Usando o dataset do projeto, crie modelos usando Árvore de Decisão e KNN para identificar se uma pessoa será adimplente ou inadimplente, realizando os pré-processamentos necessários para cada um. Utilize a metodologia de avaliação de sua preferência, mas seu modelo será avaliado em um conjunto apartado. Que conclusões você consegue tirar a partir do modelo?

Escolham apenas 5 variáveis dentro das 100+ disponíveis

In [18]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [8]:
#Importando a base de dados application_train
# Carregando o CSV
credit = pd.read_csv('C:/Users/Mari/Documents/LetsCode/Machine_Learning_I/credito-imoveis/application_train.csv')

# Mostrar as primeiras linhas
credit.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,456162,0,Cash loans,F,N,N,0,112500.0,700830.0,22738.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,134978,0,Cash loans,F,N,N,0,90000.0,375322.5,14422.5,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,3.0
2,318952,0,Cash loans,M,Y,N,0,180000.0,544491.0,16047.0,...,0,0,0,0,0.0,0.0,0.0,1.0,1.0,3.0
3,361264,0,Cash loans,F,N,Y,0,270000.0,814041.0,28971.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
4,260639,0,Cash loans,F,N,Y,0,144000.0,675000.0,21906.0,...,0,0,0,0,0.0,0.0,0.0,10.0,0.0,0.0


In [9]:
#Importando o dicionário tb!
dicionario = pd.read_csv('C:/Users/Mari/Documents/LetsCode/Machine_Learning_I/credito-imoveis/HomeCredit_columns_description.csv'
                         ,encoding='latin1')
dicionario.head(40)

,Unnamed: 0,Table,Row,Description,Special
0,1,application_{train|test}.csv,SK_ID_CURR,ID of loan in our sample,NaN
1,2,application_{train|test}.csv,TARGET,Target variable (1 - client with payment diffi...,NaN
2,5,application_{train|test}.csv,NAME_CONTRACT_TYPE,Identification if loan is cash or revolving,NaN
3,6,application_{train|test}.csv,CODE_GENDER,Gender of the client,NaN
4,7,application_{train|test}.csv,FLAG_OWN_CAR,Flag if the client owns a car,NaN
5,8,application_{train|test}.csv,FLAG_OWN_REALTY,Flag if client owns a house or flat,NaN
6,9,application_{train|test}.csv,CNT_CHILDREN,Number of children the client has,NaN
7,10,application_{train|test}.csv,AMT_INCOME_TOTAL,Income of the client,NaN
8,11,application_{train|test}.csv,AMT_CREDIT,Credit amount of the loan,NaN
9,12,application_{train|test}.csv,AMT_ANNUITY,Loan annuity,NaN


In [12]:
#Criando um novo dataset com as variáveis que escolho + target
credit1 =  credit[['AMT_INCOME_TOTAL', 
            'CNT_CHILDREN', 
            'DAYS_EMPLOYED',
            'DAYS_BIRTH',
            'REGION_RATING_CLIENT',
            'TARGET']]

credit1

,AMT_INCOME_TOTAL,CNT_CHILDREN,DAYS_EMPLOYED,DAYS_BIRTH,REGION_RATING_CLIENT,TARGET
0,112500.0,0,-813,-8676,2,0
1,90000.0,0,-223,-13583,2,0
2,180000.0,0,-6202,-13993,2,0
3,270000.0,0,365243,-22425,1,0
4,144000.0,0,-2763,-18839,2,0
...,...,...,...,...,...,...
246003,270000.0,1,-4333,-15992,2,0
246004,180000.0,0,365243,-23798,3,0
246005,112500.0,1,-1982,-14793,1,1
246006,202500.0,3,-2840,-14418,2,1


In [14]:
#Breve descritiva
credit1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 246008 entries, 0 to 246007
Data columns (total 6 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   AMT_INCOME_TOTAL      246008 non-null  float64
 1   CNT_CHILDREN          246008 non-null  int64  
 2   DAYS_EMPLOYED         246008 non-null  int64  
 3   DAYS_BIRTH            246008 non-null  int64  
 4   REGION_RATING_CLIENT  246008 non-null  int64  
 5   TARGET                246008 non-null  int64  
dtypes: float64(1), int64(5)
memory usage: 11.3 MB


In [15]:
# Percentual de valores nulos
(credit1.isna().sum()/len(credit1))*100

AMT_INCOME_TOTAL        0.0
CNT_CHILDREN            0.0
DAYS_EMPLOYED           0.0
DAYS_BIRTH              0.0
REGION_RATING_CLIENT    0.0
TARGET                  0.0
dtype: float64

In [19]:
#Define as variáveis explicativas e o target
X =  credit1[['AMT_INCOME_TOTAL', 
            'CNT_CHILDREN', 
            'DAYS_EMPLOYED',
            'DAYS_BIRTH',
            'REGION_RATING_CLIENT']]

y = credit1['TARGET']

In [22]:
X.head()

,AMT_INCOME_TOTAL,CNT_CHILDREN,DAYS_EMPLOYED,DAYS_BIRTH,REGION_RATING_CLIENT
0,112500.0,0,-813,-8676,2
1,90000.0,0,-223,-13583,2
2,180000.0,0,-6202,-13993,2
3,270000.0,0,365243,-22425,1
4,144000.0,0,-2763,-18839,2


In [23]:
y.head()

0    0
1    0
2    0
3    0
4    0
Name: TARGET, dtype: int64

In [25]:
# Separando os dados entre treino e teste
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size = 0.3,
                                                    random_state = 42)



In [26]:
# Shape das bases de treino e teste
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(172205, 5) (73803, 5) (172205,) (73803,)


In [27]:
##Precisa normalizar os dados, porque estão em escalas muito diferentes!
# Instanciar a função
scaler = StandardScaler()

In [28]:
# Transformar os dados
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)

In [29]:
X_train_std

array([[-0.11280471,  0.81393239, -0.46673284,  0.22282842,  1.86452206],
       [-0.2616478 , -0.57692077, -0.45712394, -0.6287266 , -0.09960648],
       [ 0.33372456,  3.59563871, -0.48602129,  0.16943436, -2.06373503],
       ...,
       [-0.11280471, -0.57692077, -0.46052945, -0.91105054, -0.09960648],
       [ 0.25930301,  0.81393239, -0.47547976,  0.27782659, -0.09960648],
       [ 2.41752779, -0.57692077, -0.46871821,  0.61766946, -0.09960648]])

In [30]:
X_test_std

array([[-0.11280471,  0.81393239, -0.45566847,  1.30766735, -0.09960648],
       [-0.15745763,  0.81393239, -0.4575408 ,  1.13877713, -0.09960648],
       [-0.11280471, -0.57692077, -0.4564598 ,  1.207754  , -0.09960648],
       ...,
       [ 0.18488147,  0.81393239, -0.45426953, -0.73459808, -0.09960648],
       [-0.03838316, -0.57692077, -0.47783253,  0.03583464,  1.86452206],
       [-0.11280471, -0.57692077, -0.49320677,  0.21159963, -0.09960648]])

In [37]:
# Define k = 3 vizinhos mais próximos
knn3 = KNeighborsClassifier(3)

In [38]:
# Ajusta o modelo
knn3.fit(X_train_std, y_train)

KNeighborsClassifier(n_neighbors=3)

In [40]:
# Aplica o modelo ajustado nos dados teste para predizer a variável resposta
y_pred3 = knn3.predict(X_test_std)

In [43]:
#Calcula métrica de avaliação
acuracia3 = accuracy_score(y_test, y_pred3)
acuracia3

0.9023616926141214

In [45]:
#determinando que K = 1 
knn1 = KNeighborsClassifier(1)
knn1.fit(X_train_std, y_train)
y_pred1 = knn1.predict(X_test_std)
acuracia1 = accuracy_score(y_test, y_pred1)
acuracia1

0.852417923390648

In [46]:
#determinando que K = 5 
knn5 = KNeighborsClassifier(5)
knn5.fit(X_train_std, y_train)
y_pred5 = knn5.predict(X_test_std)
acuracia5 = accuracy_score(y_test, y_pred5)
acuracia5

0.9141633808923757

In [47]:
#determinando que K = 7 
knn7 = KNeighborsClassifier(7)
knn7.fit(X_train_std, y_train)
y_pred7 = knn7.predict(X_test_std)
acuracia7 = accuracy_score(y_test, y_pred7)
acuracia7

0.9175778762380933

In [ ]:
#Conclusão: aparentemente, não faz muito sentido usar valores maiores para K, pois o ganho na acurácia não é muito significativo
#Sendo assim, um bom valor para K seria 3 ou 5